## Objectif
Ce notebook illustre un cas d'usage géospatial synthétique analysé avec `morpho-analytics`.


### Imports


In [ ]:
from pathlib import Path
import json

import numpy as np
from IPython.display import Image, display

from morpho_analytics.track import track, metrics, save_overlay_png


### Générer la série raster synthétique
Nous créons un cube `(T,H,W)` représentant des hotspots évolutifs et du bruit.


In [ ]:
data_dir = Path.cwd()
T, H, W = 15, 128, 128
series_path = data_dir / 'geo_series.npy'

rng = np.random.default_rng(42)
centers = [(32, 40), (80, 70), (96, 24)]
radii = [8.0, 10.0, 6.0]
ys, xs = np.mgrid[0:H, 0:W]
series = np.zeros((T, H, W), dtype=np.float32)
for t in range(T):
    arr = np.zeros((H, W), dtype=np.float32)
    for (cy, cx), r in zip(centers, radii):
        rr = r * (1.0 + 0.1 * np.sin(2 * np.pi * (t / 6.0)))
        d2 = (xs - cx) ** 2 + (ys - cy) ** 2
        arr += np.exp(-d2 / (2.0 * (rr ** 2)))
    arr += 0.05 * rng.standard_normal((H, W)).astype(np.float32)
    arr -= arr.min()
    m = arr.max()
    if m > 0:
        arr /= m
    series[t] = arr

np.save(series_path, series)
series.shape


### Tracking des hotspots
Application directe de `track()` sur la série synthétique.


In [ ]:
tracks = track(series, threshold=None, connectivity=4, min_area=5)
tracks


### Calcul et inspection des métriques


In [ ]:
geo_report = metrics(tracks)
print(json.dumps(geo_report, indent=2))


### Sauvegarder le rapport JSON


In [ ]:
report_path = data_dir / 'geo_report.json'
report_path.write_text(json.dumps(geo_report, indent=2))
report_path


### Générer et afficher l'overlay synthétique


In [ ]:
overlay_path = data_dir / 'geo_overlay.png'
save_overlay_png(series, tracks.labels, str(overlay_path))
display(Image(filename=str(overlay_path)))


### TODO
- Remplacer ce jeu synthétique par un raster public (COPERNICUS, Sentinel, etc.).
- Gérer la projection/échelle géographique selon la source réelle.
